In [2]:
import pandas as pd
import evaluation_helper as eval

# Read in data

features = pd.read_csv('./data/answer.csv', sep=';')
#features = pd.read_csv('../data/answer.csv', sep=';')

features.describe()

/home/dave/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user,place_asked,place_answered,type,response_time,place_map,language,ip_id
count,1.008730e+07,1.008730e+07,1.008730e+07,9.805599e+06,1.008730e+07,1.008730e+07,9.590318e+06,1.008730e+07,9.313760e+06
mean,5.057236e+06,5.992753e+04,4.918974e+02,4.880717e+02,1.158527e+00,1.388761e+07,1.872767e+02,5.638374e-02,1.778498e+04
std,2.913090e+06,3.574237e+04,4.610208e+02,4.598886e+02,3.652348e-01,1.734263e+08,6.405243e+01,2.565005e-01,1.284761e+04
min,2.407000e+03,7.500000e+01,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,5.800000e+01,0.000000e+00,2.000000e+00
25%,2.535713e+06,2.827000e+04,1.280000e+02,1.260000e+02,1.000000e+00,2.559000e+03,1.150000e+02,0.000000e+00,6.769000e+03
50%,5.057811e+06,6.134200e+04,2.170000e+02,2.160000e+02,1.000000e+00,3.733000e+03,2.270000e+02,0.000000e+00,1.567400e+04
75%,7.579838e+06,8.926200e+04,9.830000e+02,9.780000e+02,1.000000e+00,6.161000e+03,2.300000e+02,0.000000e+00,2.776700e+04
max,1.010184e+07,1.242560e+05,1.473000e+03,1.473000e+03,2.000000e+00,4.294967e+09,2.320000e+02,2.000000e+00,4.528100e+04


In [3]:
# Parsing labels
import numpy as np
questions = np.array(features['place_asked'])


In [4]:
len(np.unique(questions))

1458

In [6]:
questions = np.array(features['place_asked'])
answers = np.array(features['place_answered'])
labels = np.equal(questions, answers)
labels = list(map(lambda x: 1 if x else 0, labels))
labels = np.array(labels)


In [10]:
features['correct'] = labels
features.sample(10)

,id,user,place_asked,place_answered,type,inserted,response_time,place_map,language,options,ip_country,ip_id,correct
8850478,8864944,108356,1246,1246.0,1,2015-04-17 18:26:09,2627,79.0,0,[],CZ,35977.0,1
9338462,9352928,114741,160,160.0,1,2015-04-29 22:48:23,3462,231.0,0,[],CZ,42297.0,1
752032,762559,7497,660,660.0,1,2014-04-07 21:09:32,1775,79.0,0,"[720, 660, 652]",NaN,NaN,1
9821815,9836344,120933,241,241.0,1,2015-05-14 15:15:58,1957,79.0,0,[],CZ,38471.0,1
5938906,5953125,70254,64,64.0,1,2015-01-19 16:12:47,1986,230.0,0,[],CZ,27056.0,1
6899864,6914224,81922,28,28.0,1,2015-02-17 18:46:45,4099,195.0,0,[],CZ,31176.0,1
9807019,9821548,118173,860,860.0,1,2015-05-13 21:28:38,6642,230.0,0,[],CZ,13665.0,1
5273954,5288113,63736,541,NaN,1,2015-01-06 14:32:12,4863,195.0,1,[],CZ,20571.0,0
4421332,4435459,54583,40,40.0,1,2014-12-03 21:26:03,3289,195.0,0,[],CZ,20493.0,1
376314,383640,3448,186,186.0,1,2014-02-23 19:21:12,3681,NaN,0,[],NaN,NaN,1


In [31]:
ids = [i for i in range(224)]
filtered = features[features['place_asked'].isin(ids)]
grouped = filtered.groupby('place_asked')['correct'].agg([['min','mean']])


In [30]:
grouped.to_csv("~/test.csv")